# PyTorch 모델 저장, 양자화, ONNX 변환 

In [1]:
import torch
import torch.nn as nn
import torch.quantization
import onnx
import onnxruntime as ort
import numpy as np
import time
from pathlib import Path

# 저장 디렉토리 생성
save_dir = Path('mlp_models')
save_dir.mkdir(exist_ok=True)

# SimpleMLP 모델 정의

In [2]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=256, num_classes=10):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 학습 

In [3]:
# 데이터 로더 준비 (MNIST 예시)
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [4]:
# 모델 학습
model = SimpleMLP()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print("🚀 학습 시작...\n")
for epoch in range(5):
    model.train()  # ✅ 학습 모드 설정
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # 통계
        epoch_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y.size(0)
        correct += predicted.eq(y).sum().item()
    
    # 에폭별 결과 출력
    avg_loss = epoch_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f"Epoch {epoch+1}/5 - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("\n✅ 학습 완료!")

🚀 학습 시작...

Epoch 1/5 - Loss: 0.2467, Accuracy: 92.67%
Epoch 2/5 - Loss: 0.1159, Accuracy: 96.47%
Epoch 3/5 - Loss: 0.0882, Accuracy: 97.19%
Epoch 4/5 - Loss: 0.0688, Accuracy: 97.81%
Epoch 5/5 - Loss: 0.0598, Accuracy: 97.99%

✅ 학습 완료!


# 1️⃣ 모델 저장

In [5]:
# 방법 1: 가중치만 저장 (권장 ✅)
weights_path = save_dir / 'model_weights.pth'
torch.save(model.state_dict(), weights_path)
print(f"✅ 가중치 저장 완료: {weights_path}")
print(f"   파일 크기: {weights_path.stat().st_size / 1024:.2f} KB\n")

# 방법 2: 모델 전체 저장
full_model_path = save_dir / 'model.pth'
torch.save(model, full_model_path)
print(f"📦 전체 모델 저장 완료: {full_model_path}")
print(f"   파일 크기: {full_model_path.stat().st_size / 1024:.2f} KB\n")

# 방법 3: 체크포인트 저장 (학습 재개용)
checkpoint_path = save_dir / 'checkpoint.pth'
torch.save({
    'epoch': 5,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': avg_loss,
}, checkpoint_path)
print(f"💾 체크포인트 저장 완료: {checkpoint_path}")
print(f"   파일 크기: {checkpoint_path.stat().st_size / 1024:.2f} KB")


✅ 가중치 저장 완료: mlp_models\model_weights.pth
   파일 크기: 797.52 KB

📦 전체 모델 저장 완료: mlp_models\model.pth
   파일 크기: 798.51 KB

💾 체크포인트 저장 완료: mlp_models\checkpoint.pth
   파일 크기: 2390.80 KB


# 2️⃣ 양자화 (Quantization)
## Dynamic Quantization (동적 양자화) 중 Eager Mode / Post-Training Quantization
### 모델 크기 감소 및 추론 속도 향상

In [6]:
# 동적 양자화 (Dynamic Quantization)
model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {nn.Linear},  # 양자화할 레이어 타입
    dtype=torch.qint8
)

# 양자화 모델 저장
quantized_path = save_dir / 'model_quantized_weights.pth'
torch.save(quantized_model.state_dict(), quantized_path)
print(f"⚡ 양자화 모델 가중치 저장 완료: {quantized_path}")
print(f"   파일 크기: {quantized_path.stat().st_size / 1024:.2f} KB")
print(f"   압축률: {weights_path.stat().st_size / quantized_path.stat().st_size:.2f}x\n")

full_quantized_path = save_dir / 'model_quantized.pth'
torch.save(quantized_model, full_quantized_path)
# torch.save(quantized_model, str(quantized_path) + '_full')
# torch.save(quantized_model, quantized_path.with_name(quantized_path.stem + '_full.pth'))
print(f"⚡ 양자화 모델 전체 저장 완료: {full_quantized_path}")
print(f"   파일 크기: {full_quantized_path.stat().st_size / 1024:.2f} KB")
print(f"   압축률: {full_model_path.stat().st_size / full_quantized_path.stat().st_size:.2f}x\n")


⚡ 양자화 모델 가중치 저장 완료: mlp_models\model_quantized_weights.pth
   파일 크기: 203.54 KB
   압축률: 3.92x

⚡ 양자화 모델 전체 저장 완료: mlp_models\model_quantized.pth
   파일 크기: 203.98 KB
   압축률: 3.91x



FP32 → INT8 이론적 최대 압축률 ≈ 4x

결과: 3.9
→ 거의 이론적 최대치

# 3️⃣ ONNX 변환

In [7]:
# ONNX로 변환
model.eval()
dummy_input = torch.randn(1, 1, 28, 28)  # MNIST 입력 크기
onnx_path = save_dir / 'model.onnx'

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamo=False,  # ✅ 단일 파일로 저장
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)


print(f"🔄 ONNX 변환 완료: {onnx_path}")
print(f"   파일 크기: {onnx_path.stat().st_size / 1024:.2f} KB\n")

# ONNX 모델 검증
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("✅ ONNX 모델 검증 완료")



🔄 ONNX 변환 완료: mlp_models\model.onnx
   파일 크기: 796.04 KB

✅ ONNX 모델 검증 완료


In [ ]:
# 모델 로드 예시
# 가중치 로드 (권장)
# weights_path=save_dir/'model_weights.pth'
loaded_model = SimpleMLP()
loaded_model.load_state_dict(torch.load(weights_path))
loaded_model.eval()
print("✅ 가중치 로드 완료\n")

from torch.serialization import add_safe_globals
# SimpleMLP 클래스 정의가 현재 코드에 있어야 함
add_safe_globals([SimpleMLP])
# 전체
full_model_path = save_dir / 'model.pth' 
model = torch.load(full_model_path)
# 체크포인트 로드 (학습 재개용)
checkpoint = torch.load(checkpoint_path)
resume_model = SimpleMLP()
resume_model.load_state_dict(checkpoint['model_state_dict'])
resume_optimizer = torch.optim.Adam(resume_model.parameters())
resume_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
resume_epoch = checkpoint['epoch']
resume_loss = checkpoint['loss']
print(f"✅ 체크포인트 로드 완료 (Epoch {resume_epoch}, Loss {resume_loss:.4f})\n")

✅ 가중치 로드 완료



UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch.nn.modules.linear.Linear was not an allowed global by default. Please use `torch.serialization.add_safe_globals([torch.nn.modules.linear.Linear])` or the `torch.serialization.safe_globals([torch.nn.modules.linear.Linear])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to lea